In [11]:
import pandas as pd
import json
import requests
import datetime as dt
from datetime import date, timedelta, datetime
from tqdm.notebook import tqdm
import time

# Function to make the start dates list 
def days(start, end, step=timedelta(days=90)):
    curr = start
    while curr <= end:
        yield curr
        curr += step

# Function to select day type


In [181]:
# Inrix live data (1min)
url = 'http://inrixfreiburg.traffgoroad.com/inrix_freiburg.json'
username = 'arup'
password = 'RJXvRaFMk:'

response=requests.get(url, auth=(username, password))
response_json=response.json()

data_segment=pd.DataFrame()

for result in response_json['result']['segmentSpeeds']:
    segments=result['segments']
    data=pd.DataFrame(segments)
    data['date']=datetime.today().strftime('%Y-%m-%d')
    data['time']=dt.datetime.now().strftime('%H:%M')
    data_segment=pd.concat([data_segment,data])

data.to_csv(r'C:\Users\Ibrahem.Alsalamh\OneDrive - Arup\Freiburg smart city\traffic data\inrix_data_new.csv')

In [7]:
# Create the date list
starting_date=date(2020, 1, 1)
date_list=list(days(starting_date, dt.date.today()))

In [ ]:
# POSTtraffic snapshot Telraam to get a list of segment ids
data_segment=pd.DataFrame()
for start_date in tqdm(date_list):
  x = 1
  time.sleep(x)
  url = "https://telraam-api.net/v1/reports/traffic_snapshot"
  payload = "{\r\n    \"time\":\"" +str(start_date)+ " 00:00:00Z\",\r\n    \"contents\":\"full\",\r\n    \"area\":\"7.8359048,47.9963918,10\"\r\n}\r\n"
  headers = {'X-Api-Key': '9bDufSS8AE2NAOT2R5YtR9vutyukJlZh6ZQyhTnQ'}
  response = requests.request("POST", url, headers=headers, data=payload,)
  response_json=response.json()
    
  for result in response_json['features']:
    result=result['properties']
    data=pd.DataFrame(result,index=[0])
    data_segment=pd.concat([data_segment,data])

segment_ids_list=data_segment['segment_id'].unique().tolist()

In [30]:
# POSTtraffic Telraam to get historical data
date_list=list(days(date(2020, 1, 1), dt.date.today()))
data_repot=pd.DataFrame()

for segment_id in tqdm(segment_ids_list):
    for start_date in tqdm(date_list):
        x = 2
        time.sleep(x)
        end_date=start_date+ dt.timedelta(days=89)
        if end_date <= dt.date.today():
            payload = "{\r\n  \"level\": \"segments\",\r\n  \"format\": \"per-hour\",\r\n  \"id\": \""+str(segment_id)+"\",\r\n  \"time_start\": \"" +str(start_date)+ " 00:00:00Z\",\r\n  \"time_end\": \""+ str(end_date)+ " 23:00:00Z\"\r\n}"
            url = "https://telraam-api.net/v1/reports/traffic"
            headers = {'X-Api-Key': '9bDufSS8AE2NAOT2R5YtR9vutyukJlZh6ZQyhTnQ'}
            response = requests.request("POST", url, headers=headers, data=payload)
            response_json=response.json()

            for result in response_json['report']:
                data=pd.DataFrame([result])
                data_repot=pd.concat([data_repot,data])

start_date = end_date+dt.timedelta(days=-89)            
 end_date2 = dt.date.today()
 payload = "{\r\n  \"level\": \"segments\",\r\n  \"format\": \"per-hour\",\r\n  \"id\": \""+str(segment_id)+"\",\r\n  \"time_start\": \"" +str(start_date)+ " 00:00:00Z\",\r\n  \"time_end\": \""+ str(end_date)+ " 23:00:00Z\"\r\n}"
 url = "https://telraam-api.net/v1/reports/traffic"
 headers = {'X-Api-Key': '9bDufSS8AE2NAOT2R5YtR9vutyukJlZh6ZQyhTnQ'}
 response = requests.request("POST", url, headers=headers, data=payload)
 response_json=response.json()

 for result in response_json['report']:
    data=pd.DataFrame([result])
    data_repot=pd.concat([data_repot,data])
        
 data_repot.to_csv(r'C:\Users\Ibrahem.Alsalamh\OneDrive - Arup\Freiburg smart city\traffic data\telraam_data_freiburg_complete.csv')
       
    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 22)

In [25]:
data_repot

,instance_id,segment_id,date,interval,uptime,heavy,car,bike,pedestrian,heavy_lft,...,car_rgt,bike_lft,bike_rgt,pedestrian_lft,pedestrian_rgt,direction,car_speed_hist_0to70plus,car_speed_hist_0to120plus,timezone,v85
0,-1,9000001395,2021-02-24T12:00:00.000Z,hourly,0.086667,0.000000,11.538462,11.538462,0.000000,0.000000,...,0.000000,0.000000,11.538462,0.000000,0.000000,1,"[0.0, 100.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 100.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",Europe/Berlin,11.5
0,-1,9000001395,2021-02-24T14:00:00.000Z,hourly,0.409722,0.000000,602.847458,256.271186,26.847458,0.000000,...,324.610169,114.711864,141.559322,7.322034,19.525424,1,"[22.6720647773, 64.3724696356, 12.5506072874, ...","[8.0971659919, 14.5748987854, 33.1983805668, 3...",Europe/Berlin,17.0
0,-1,9000001395,2021-02-24T15:00:00.000Z,hourly,0.063611,0.000000,534.497817,31.441048,0.000000,0.000000,...,267.248908,0.000000,31.441048,0.000000,0.000000,1,"[70.5882352941, 23.5294117647, 0.0, 2.94117647...","[29.4117647059, 41.1764705882, 20.5882352941, ...",Europe/Berlin,11.0
0,-1,9000001395,2021-02-24T16:00:00.000Z,hourly,0.074167,0.000000,822.471910,134.831461,0.000000,0.000000,...,404.494382,13.483146,121.348315,0.000000,0.000000,1,"[72.131147541, 26.2295081967, 1.6393442623, 0....","[19.6721311475, 52.4590163934, 24.5901639344, ...",Europe/Berlin,10.0
0,-1,9000001395,2021-02-24T19:00:00.000Z,hourly,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Europe/Berlin,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,-1,9000003844,2022-07-20T16:00:00.000Z,hourly,0.351111,45.569620,444.303797,600.949367,59.810127,8.544304,...,207.911392,296.202532,304.746835,28.481013,31.329114,1,"[16.0256410256, 75.0, 5.7692307692, 0.0, 0.641...","[10.8974358974, 5.1282051282, 40.3846153846, 3...",Europe/Berlin,16.5
0,-1,9000003844,2022-07-20T17:00:00.000Z,hourly,0.697778,32.961783,299.522293,260.828025,28.662420,5.732484,...,127.547771,153.343949,107.484076,8.598726,20.063694,1,"[6.2200956938, 60.2870813397, 28.7081339713, 1...","[4.3062200957, 1.9138755981, 22.4880382775, 37...",Europe/Berlin,22.0
0,-1,9000003844,2022-07-20T18:00:00.000Z,hourly,0.650556,7.685739,162.937660,146.029035,13.834330,0.000000,...,56.874466,86.080273,59.948762,3.074295,10.760034,1,"[1.8867924528, 55.6603773585, 33.0188679245, 2...","[1.8867924528, 0.0, 15.0943396226, 40.56603773...",Europe/Berlin,24.0
0,-1,9000003844,2022-07-20T19:00:00.000Z,hourly,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Europe/Berlin,None


In [ ]:
# we stil need the coordinate, add day type, fix the speed buckets and do the merging with the osm data 
# we need to check the segment id if we have all counters 
# we need to understand why this is the last time 2022-07-20T20:00:00.000Z

In [110]:
# POSTtraffic Telraam to get historical data 
url = "https://telraam-api.net/v1/reports/traffic"

headers = {
  'X-Api-Key': '9bDufSS8AE2NAOT2R5YtR9vutyukJlZh6ZQyhTnQ'
}

response = requests.request("POST", url, headers=headers, data=payload)
response_json=response.json()

data_repot=pd.DataFrame()
for result in response_json['report']:
    #print(result)
    data=pd.DataFrame([result])
    data_repot=pd.concat([data_repot,data])

data_repot
data_repot.to_csv(r'C:\Users\Ibrahem.Alsalamh\OneDrive - Arup\Freiburg smart city\traffic data\telraam_data_postTraffic.csv')